# Modeling
Esse Notebook visa apresentar a etapa de definição do modelo de machine learning usado para fazer a predição da ```ICU```.<br>
Após a etapa de *Feature Engineering* os dados estão prontos para serem usados de maneira a auxiliar a classificação.<br>

<a name="sumario"></a>
# Sumário
<!--ts-->
   * [Importe das Bibliotecas](#import)
   * [Leitura dos dados](#leitura)
   * [Escalar o AGE_PERCENTIL](#age)
   * [Features com alta correlação](#corr)
<!--te-->

<a name="import"></a>
# Importe das Bibliotecas

## Bibliotecas Externas

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

## Bibliotecas Internas

In [2]:
import sys
sys.path.insert(1, "../src/")
from supervised import LazyClassifier
from modeling import run_model_cv

-----------------
Retornar ao [Sumário](#sumario)

<a name="leitura"></a>
# Leitura dos dados
Faz a leitura dos dados que serão usados para a modelagem.<br>
**```df : pd.DataFrame```** é o DataFrame contendo os dados que já passaram pela etapa de *feature engineering*.

In [3]:
# Leitura do raw data desse  projeto no Github
df = pd.read_csv('../data/processed/df_featured.csv', index_col='Unnamed: 0')
df.index.names = ['PATIENT_VISIT_IDENTIFIER']

# Imprime na tela todas as colunas em vez de as 10 primeiras e 10 ultimas
pd.set_option('max_columns', df.shape[1])
df.head()

,AGE_ABOVE65,AGE_SCALED,DISEASE GROUPING 1,DISEASE GROUPING 3,DISEASE GROUPING 5,HTN,IMMUNOCOMPROMISED,ALBUMIN,BE_VENOUS,FFA,GGT,GLUCOSE,HEMOGLOBIN,LACTATE,LINFOCITOS,NEUTROPHILES,P02_VENOUS,PCR,SAT02_VENOUS,SODIUM,DIMER,BLOODPRESSURE_DIASTOLIC_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,BLOODPRESSURE_SISTOLIC_MEDIAN,OXYGEN_SATURATION_MEDIAN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_MAX_MEDIAN_RESIDUAL,HEART_RATE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_SISTOLIC_DIFF,ICU
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0.11,0.00,0.00,0.00,1.00,0.00,0.61,-1.00,-0.74,-0.96,-0.85,-0.02,1.00,-0.66,-0.79,-0.70,-1.00,0.35,-0.14,-0.97,0.19,-0.39,-0.29,-0.28,0.84,-0.44,-0.29,-0.18,-0.02,-0.67,0.84,-1.00,0
1,0,0.00,0.00,0.00,0.00,0.00,0.00,0.61,-1.00,-0.74,-0.99,-0.89,-0.17,1.00,-0.58,-0.89,-0.70,-0.95,0.35,-0.03,-0.98,0.19,-0.46,-0.11,-0.52,1.00,-0.32,-0.36,-0.18,-0.02,-0.57,1.00,-1.00,0
2,0,0.22,0.00,0.00,0.00,0.00,0.00,0.61,-0.86,-0.74,-0.96,-0.21,0.07,1.00,-0.82,-0.95,-0.66,-0.96,0.48,0.07,-0.98,0.09,-0.53,0.40,-0.38,0.68,-0.32,-0.43,-0.25,-0.02,-0.57,0.68,-1.00,0
3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.61,-1.00,-0.74,-0.96,-0.89,-0.27,1.00,-0.48,-0.92,-0.70,-0.98,0.35,0.07,-0.99,-0.16,-0.53,-0.14,-0.69,0.95,-0.25,-0.43,-0.42,-0.02,-0.51,0.95,-1.00,0
4,1,0.78,0.00,0.00,1.00,0.00,1.00,0.00,-0.88,-0.68,-0.75,-0.81,-0.48,-0.79,-0.80,-0.89,-0.92,-0.44,-0.70,0.07,-0.98,0.90,-0.17,0.14,-0.34,0.79,-0.04,-0.43,0.32,-0.02,-0.33,0.79,-1.00,1


# Métrica de avaliação:
A métrica de avaliação utilizada para determinar o modelo de machine learning será o **ROC AUC**.<br>
* **ROC** do inglês *Receiver Operating Characteristics* é uma curva plotada entre os eixos *Taxa de Verdadeiros Positivos* e a *Taxa de Falsos Positivos*. Essa métrica mede o quanto o modelo consegue separar o binômio de saída entre suas duas classes (```ICU = 0``` e ```ICU = 1```).
* **AUC** do inglês *Area Under the Curve* é a maneira de resumir a curva **ROC** em um único valor real, de maneira a facilitar a comparação com outros modelos. Seria o mesmo que calcular a inegral do ROC<br>
* **ROC AUC**<br>
    * O valor será próximo de 0.5 se o modelo não tiver capacidade de separar as variáveis de saída em suas respectivas classes. Imagine que o modelo Tem 50% de verdadeiro positivo e 50% de falso positivo, ou seja, acerta em 50% das tentativas.
    * Quanto mais o valor se aproxima de 1.0, mais capacidade o modelo tem de separar os dois grupos, ou classes, acertando muito mais os Verdadeiros Positivos que errando.
    * O inverso também é verdadeiro. Quanto mais o valor se aproxima de 0, significa que o modelo também tem a capacidade de separar o binômio de saída em dois grupos, só que nesse caso, errando muito, pois teria um valor de Falsos Verdadeiros muito grande. Como a saída tem 2 estados, sendo 0 ou 1, poderia ser usado o inverso (NOT) da resposta e ainda assim o modelo acertaria bastante.

# Dummy Classifier
Essa etapa visa criar um modelo genérico que faz predições dos dados de entrada usando regras bem simples. Ter um modelo "bobo" que usa uma regra simples é importante para avaliar o modelo final, pois se espera do modelo a ser implementado que ao menos seja melhor que escolher aleatóriamente entre as alternativas de saída, ou então "chutar" a alternativa de saída mais comum.

In [6]:
run_model_cv('DummyClassifier', DummyClassifier, df, 7, 10)

DummyClassifier: AUC Mean: 0.5, AUC Std: 0.0, AUC CI: 0.5 - 0.5


Vemos que o DummyClassifier encontrou um ROC AUC Médio de **0.5**, logo, essa deve ser a meta inicial a ser batida.

In [41]:
run_model_cv('RandomForestClassifier', RandomForestClassifier, df, 70, 5)

RandomForestClassifier: AUC Mean: 0.81, AUC Std: 0.219, AUC CI: 0.37 - 1.25


In [42]:
run_model_cv('RandomForestClassifier', RandomForestClassifier, df, 5, 5)

RandomForestClassifier: AUC Mean: 0.8, AUC Std: 0.051, AUC CI: 0.7 - 0.9


In [45]:
y = df["ICU"]
X = df.drop(["ICU"], axis=1)
model_list = []
for _ in range(50):
    print(_)
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, test_size=0.10)
    clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
    models,predictions = clf.fit(X_train, X_test, y_train, y_test)
    model_list.append(models['ROC AUC'])

0


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 23.58it/s]


[19:10:50] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 23.01it/s]


[19:10:52] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2


...

[19:11:53] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
48


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 23.70it/s]


[19:11:54] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
49


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 25.57it/s]

[19:11:55] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [53]:
pd.DataFrame(model_list).mean().sort_values(ascending=False)

Model
RandomForestClassifier          0.72
NuSVC                           0.72
SVC                             0.72
XGBClassifier                   0.71
ExtraTreesClassifier            0.71
LGBMClassifier                  0.71
BernoulliNB                     0.71
RidgeClassifierCV               0.71
BaggingClassifier               0.70
LinearDiscriminantAnalysis      0.70
RidgeClassifier                 0.70
LinearSVC                       0.70
CalibratedClassifierCV          0.70
LogisticRegression              0.70
NearestCentroid                 0.69
AdaBoostClassifier              0.67
GaussianNB                      0.67
SGDClassifier                   0.65
Perceptron                      0.65
DecisionTreeClassifier          0.65
QuadraticDiscriminantAnalysis   0.64
ExtraTreeClassifier             0.64
KNeighborsClassifier            0.62
PassiveAggressiveClassifier     0.62
LabelSpreading                  0.56
LabelPropagation                0.56
DummyClassifier                 

# Referências
* https://towardsdatascience.com/modeling-teaching-a-machine-learning-algorithm-to-deliver-business-value-ad0205ca4c86
* https://docs.microsoft.com/en-us/windows/ai/windows-ml/what-is-a-machine-learning-model
* https://medium.com/@datadrivenscience/7-stages-of-machine-learning-a-framework-33d39065e2c9
* https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc
* https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5
* https://medium.com/bio-data-blog/entenda-o-que-%C3%A9-auc-e-roc-nos-modelos-de-machine-learning-8191fb4df772